In [1]:
import pandas as pd
import numpy as np

In [40]:
tracking = pd.read_csv(r"D:\Szakdolgozat_data\nba-movement-data-master\data\csv_fix\0021500001.csv") #example game
events = pd.read_csv(r"D:\Szakdolgozat_data\nba-movement-data-master\data\events\0021500001.csv")
shots = pd.read_csv(r"D:\Szakdolgozat_data\nba-movement-data-master\data\shots\shots.csv")

In [41]:
tracking.head(5)

,game_id,event_id,game_clock,shot_clock,quarter,b_team_id,b_player_id,b_x,b_y,b_z,...,p9_team_id,p9_player_id,p9_x,p9_y,p9_z,p10_team_id,p10_player_id,p10_x,p10_y,p10_z
0,21500001,1,720.0,24.0,1,-1,-1,47.25031,26.14806,6.76567,...,1.610613e+09,203484.0,48.46352,14.55436,0.0,1.610613e+09,203083.0,47.51449,24.36448,0.0
1,21500001,1,720.0,24.0,1,-1,-1,47.16223,26.23652,6.86375,...,1.610613e+09,203484.0,48.46453,14.54789,0.0,1.610613e+09,203083.0,47.48025,24.33370,0.0
2,21500001,1,720.0,24.0,1,-1,-1,47.16051,26.18061,6.79804,...,1.610613e+09,203484.0,48.46574,14.54386,0.0,1.610613e+09,203083.0,47.44308,24.29835,0.0
3,21500001,1,720.0,24.0,1,-1,-1,47.12323,26.14186,6.84851,...,1.610613e+09,203484.0,48.46702,14.54001,0.0,1.610613e+09,203083.0,47.44450,24.29213,0.0
4,21500001,1,720.0,24.0,1,-1,-1,47.19169,26.17897,6.77785,...,1.610613e+09,203484.0,48.46976,14.53138,0.0,1.610613e+09,203083.0,47.41923,24.26615,0.0


In [53]:
min(tracking['b_z'])

0.0

In [7]:
events.head(5)

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,PLAYER2_TEAM_CITY,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION
0,21500001,0,12,0,1,8:12 PM,12:00,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
1,21500001,1,10,0,1,8:12 PM,12:00,Jump Ball Horford vs. Drummond: Tip to Ilyasova,NaN,NaN,...,Detroit,Pistons,DET,5,101141,Ersan Ilyasova,1.610613e+09,Detroit,Pistons,DET
2,21500001,2,2,42,1,8:13 PM,11:41,Horford BLOCK (1 BLK),NaN,MISS Drummond 2' Driving Layup,...,NaN,NaN,NaN,4,201143,Al Horford,1.610613e+09,Atlanta,Hawks,ATL
3,21500001,3,4,0,1,8:13 PM,11:39,Bazemore REBOUND (Off:0 Def:1),NaN,NaN,...,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
4,21500001,4,5,45,1,8:13 PM,11:37,Bazemore Out of Bounds - Bad Pass Turnover Tur...,NaN,NaN,...,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN


In [12]:
events.columns

Index(['GAME_ID', 'EVENTNUM', 'EVENTMSGTYPE', 'EVENTMSGACTIONTYPE', 'PERIOD',
       'WCTIMESTRING', 'PCTIMESTRING', 'HOMEDESCRIPTION', 'NEUTRALDESCRIPTION',
       'VISITORDESCRIPTION', 'SCORE', 'SCOREMARGIN', 'PERSON1TYPE',
       'PLAYER1_ID', 'PLAYER1_NAME', 'PLAYER1_TEAM_ID', 'PLAYER1_TEAM_CITY',
       'PLAYER1_TEAM_NICKNAME', 'PLAYER1_TEAM_ABBREVIATION', 'PERSON2TYPE',
       'PLAYER2_ID', 'PLAYER2_NAME', 'PLAYER2_TEAM_ID', 'PLAYER2_TEAM_CITY',
       'PLAYER2_TEAM_NICKNAME', 'PLAYER2_TEAM_ABBREVIATION', 'PERSON3TYPE',
       'PLAYER3_ID', 'PLAYER3_NAME', 'PLAYER3_TEAM_ID', 'PLAYER3_TEAM_CITY',
       'PLAYER3_TEAM_NICKNAME', 'PLAYER3_TEAM_ABBREVIATION'],
      dtype='object')

In [43]:
def full_to_half_full(data):
    """
    Convert full court movement to a single half court
    Parameters
    ----------
    data: pandas.DataFrame
        dataframe containing SportVU movement data that covers the entire court
    Returns
    -------
    data: pandas.DataFrame
        dataframe containing SportVU movement data that
        is converted to a single half court (x_loc < 47)
    """

    # first force all points above 47 to their half court counterparts
    # keep all original points for furhter limitations to single court
    #for ball:
    data['b_x_original'] = data['b_x']
    data['b_y_original'] = data['b_y']
    data.loc[data['b_x'] > 47,'b_y'] = data.loc[data['b_x'] > 47, 'b_y'].apply(lambda y: 50 - y)
    data.loc[data['b_x'] > 47,'b_x'] = data.loc[data['b_x'] > 47, 'b_x'].apply(lambda x: 94 - x)
    #for players:
    for num in range(1,11):
        data[f'p{num}_x_original'] = data[f'p{num}_x']
        data[f'p{num}_y_original'] = data[f'p{num}_y']
        data.loc[data[f'p{num}_x'] > 47,f'p{num}_y'] = data.loc[data[f'p{num}_x'] > 47, f'p{num}_y'].apply(lambda y: 50 - y)
        data.loc[data[f'p{num}_x'] > 47,f'p{num}_x'] = data.loc[data[f'p{num}_x'] > 47, f'p{num}_x'].apply(lambda x: 94 - x)

    return data

In [44]:
halfcourt_tracking =full_to_half_full(tracking)

In [62]:
halfcourt_tracking['b_x']

0         46.74969
1         46.83777
2         46.83949
3         46.87677
4         46.80831
            ...   
222851    28.69493
222852    28.99712
222853    29.09216
222854    29.29682
222855    29.47729
Name: b_x, Length: 222856, dtype: float64